<a href="https://colab.research.google.com/github/enhn16/Webnevel_Analysis/blob/main/01_DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 카카오페이지

## 크롤링
- 카카오페이지의 스크롤 방식의 목록 때문에 안정적인 수집이 어려움 -> 링크를 수집한 후 해당 링크의 페이지에서 상세 정보를 수집하도록 함

In [ ]:
# ===============================
# [셀 1] 환경 설정 및 라이브러리 설치
# ===============================

print("🔄 1/3: Python 라이브러리 설치 중...")
!pip install selenium pandas

import urllib.request
import json
import sys
import os

print("\n🔄 2/3: 최신 Chrome 버전 및 URL 확인 중...")
try:
    json_url = "https://googlechromelabs.github.io/chrome-for-testing/last-known-good-versions-with-downloads.json"
    response = urllib.request.urlopen(json_url)
    data = json.loads(response.read().decode('utf-8'))

    downloads = data['channels']['Stable']['downloads']

    chrome_url = next(item['url'] for item in downloads['chrome'] if item['platform'] == 'linux64')
    driver_url = next(item['url'] for item in downloads['chromedriver'] if item['platform'] == 'linux64')

except Exception as e:
    print(f"❌ 버전/URL 확인 실패: {e}")
    sys.exit()

print("\n🔄 3/3: Chrome + Driver 다운로드 및 압축 해제 중...")
try:
    !apt-get update -y > /dev/null 2>&1
    !apt-get install -y libatk-bridge2.0-0 libgtk-3-0 libasound2 libx11-xcb1 libnss3 libxss1 libxshmfence1 libgbm1 > /dev/null 2>&1

    !wget -q {driver_url} -O chromedriver.zip
    !wget -q {chrome_url} -O chrome.zip

    !unzip -o -q chromedriver.zip -d /content/
    !unzip -o -q chrome.zip -d /content/

    # 폴더 구조 평탄화 작업 (버전에 따라 폴더명이 바뀔 수 있으므로 와일드카드 사용)
    !mv /content/chromedriver-linux64/chromedriver /content/chromedriver 2>/dev/null || true
    !mv /content/chrome-linux64 /content/chrome 2>/dev/null || true

    !chmod +x /content/chromedriver
    !chmod +x /content/chrome/chrome

    print("\n설치 및 설정 완료!")
    print("-> 이제 아래의 [셀 2] 코드를 실행하세요.")
except Exception as e:
    print(f"설치 중 오류 발생: {e}")

🔄 1/3: Python 라이브러리 설치 중...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 84.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.3/510.3 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.6/131.6 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.5.0
    Uninstalling urllib3-2.5.0:
      Successfully uninstalled urllib3-2.5.0

🔄 2/3: 최신 Chrome 버전 및 URL 확인 중...

🔄 3/3: Chrome + Driver 다운로드 및 압축 해제 중...

설치 및 설정 완료!
-> 이제 아래의 [셀 2] 코드를 실행하세요.


In [ ]:
import time
import os
import random
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# ===== ⚙️ 기본 설정 =====
DRIVE_PATH = "/content/drive/MyDrive/Colab Notebooks/"
os.makedirs(DRIVE_PATH, exist_ok=True)

# 파일명도 헷갈리지 않게 현대판타지로 명시
link_output_file = os.path.join(DRIVE_PATH, "kakaopage_links.csv")
final_output_file = os.path.join(DRIVE_PATH, "kakaopage.csv")

# 수집할 칼럼 순서
COLUMNS_ORDER = ['title', 'outline', 'keyword', 'views', 'publish', 'total_chapters', 'link']

# 현대판타지(uid=120) 단일 URL 고정
TARGET_URL = "https://page.kakao.com/menu/10011/screen/84?subcategory_uid=120&sort_opt=UPDATE"


# ===== 🛠️ 드라이버 및 헬퍼 함수 =====
def start_driver():
    chrome_options = Options()
    chrome_options.binary_location = "/content/chrome/chrome"
    chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    service = Service("/content/chromedriver")
    try:
        driver = webdriver.Chrome(service=service, options=chrome_options)
        return driver
    except Exception as e:
        print(f"❌ WebDriver 시작 실패: {e}. 30초 후 재시도.")
        time.sleep(30)
        return start_driver()

def scroll_batch(driver, scrolls=10, sleep=1.5):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(scrolls):
        try:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(sleep)
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height: break
            last_height = new_height
        except: break

In [ ]:
# ===== 🚀 [1단계] 링크 수집 (최적화 버전) =====
print("\n--- 🚀 [1단계] 카카오페이지 (현대판타지) 링크 수집 시작 ---")
processed_links = set()
if os.path.exists(link_output_file):
    try:
        df_existing = pd.read_csv(link_output_file)
        processed_links = set(df_existing['link'])
        print(f"✅ 기존 링크 {len(processed_links)}개 로드 완료.")
    except: pass

driver = start_driver()
try:
    print(f"🔍 현대판타지 페이지 접속 중...")
    driver.get(TARGET_URL)
    time.sleep(5)

    # 100번 스크롤 (필요시 더 늘리세요)
    for batch_num in range(100):
        # 1. 스크롤 10번 내리기
        scroll_batch(driver, scrolls=10, sleep=3.0)

        # 🚨 스크롤 현황 출력
        print(f"⬇️ 스크롤 {batch_num + 1}회차 완료... (데이터 추출 중)")

        # 2. 핵심 최적화: 자바스크립트로 한 번에 링크/제목 싹 쓸어오기 (초고속)
        js_script = """
        let elements = document.querySelectorAll('div.text-el-60');
        let data = [];
        for (let el of elements) {
            let aTag = el.closest('a');
            if (aTag && aTag.href.includes('page.kakao.com/content/')) {
                data.push({title: el.innerText.trim(), link: aTag.href});
            }
        }
        return data;
        """
        scraped_data = driver.execute_script(js_script)

        # 3. 파이썬에서 중복 및 단행본 필터링
        new_links = []
        for item in scraped_data:
            title = item['title']
            link = item['link']

            if link not in processed_links and "단행본" not in title:
                new_links.append({'title': title, 'link': link})
                processed_links.add(link)

        # 4. 새로운 링크가 발견되었을 때만 저장
        if new_links:
            df_temp = pd.DataFrame(new_links)
            if not os.path.exists(link_output_file):
                df_temp.to_csv(link_output_file, index=False, encoding='utf-8-sig')
            else:
                df_temp.to_csv(link_output_file, mode='a', header=False, index=False, encoding='utf-8-sig')
            print(f"  💾 배치 {batch_num+1} 저장: {len(new_links)}개 신규 수집 (누적 {len(processed_links)}개)")
        else:
            # 기존 링크들만 지나치고 있을 때 띄우는 메시지
            print(f"  ⏩ 배치 {batch_num+1} 통과: 이미 수집된 구간을 지나가는 중입니다.")

finally:
    if driver: driver.quit()
    print("✅ 1단계 수집 작업 종료!")



--- 🚀 [1단계] 카카오페이지 (현대판타지) 링크 수집 시작 ---
🔍 현대판타지 페이지 접속 중...
⬇️ 스크롤 1회차 완료... (데이터 추출 중)
  💾 배치 1 저장: 245개 신규 수집 (누적 245개)
⬇️ 스크롤 2회차 완료... (데이터 추출 중)
  💾 배치 2 저장: 122개 신규 수집 (누적 367개)
⬇️ 스크롤 3회차 완료... (데이터 추출 중)
  💾 배치 3 저장: 141개 신규 수집 (누적 508개)
⬇️ 스크롤 4회차 완료... (데이터 추출 중)
  💾 배치 4 저장: 147개 신규 수집 (누적 655개)
⬇️ 스크롤 5회차 완료... (데이터 추출 중)
  💾 배치 5 저장: 117개 신규 수집 (누적 772개)
⬇️ 스크롤 6회차 완료... (데이터 추출 중)
  💾 배치 6 저장: 121개 신규 수집 (누적 893개)
⬇️ 스크롤 7회차 완료... (데이터 추출 중)
  💾 배치 7 저장: 118개 신규 수집 (누적 1011개)
⬇️ 스크롤 8회차 완료... (데이터 추출 중)
  💾 배치 8 저장: 143개 신규 수집 (누적 1154개)
⬇️ 스크롤 9회차 완료... (데이터 추출 중)
  💾 배치 9 저장: 141개 신규 수집 (누적 1295개)
⬇️ 스크롤 10회차 완료... (데이터 추출 중)
  💾 배치 10 저장: 113개 신규 수집 (누적 1408개)
⬇️ 스크롤 11회차 완료... (데이터 추출 중)
  💾 배치 11 저장: 131개 신규 수집 (누적 1539개)
⬇️ 스크롤 12회차 완료... (데이터 추출 중)
  💾 배치 12 저장: 119개 신규 수집 (누적 1658개)
⬇️ 스크롤 13회차 완료... (데이터 추출 중)
  💾 배치 13 저장: 130개 신규 수집 (누적 1788개)
⬇️ 스크롤 14회차 완료... (데이터 추출 중)
  💾 배치 14 저장: 143개 신규 수집 (누적 1931개)
⬇️ 스크롤 15회차 완료... (데이터 추출 중)
  💾 배치 15 저장: 145개 신규 수집

In [ ]:
# ===== 🚀 [2단계] 상세 정보 수집 =====
def get_novel_details(driver, base_url):
    details = {col: None for col in COLUMNS_ORDER}
    details['link'] = base_url
    wait = WebDriverWait(driver, 10)

    try:
        # 1. 홈 탭 접속
        driver.get(base_url + "?tab_type=overview")
        time.sleep(random.uniform(2.0, 3.0)) # 로딩 대기 시간

        # 페이지 전체 소스를 미리 가져와서 정규식 탐색에 활용
        html_source = driver.page_source

        # [기본 정보 수집]
        try: details['title'] = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "span[class*='font-large3-bold']"))).text.strip()
        except: pass

        try: details['views'] = driver.find_element(By.XPATH, "//img[@alt='열람자']/following-sibling::span").text.strip()
        except: pass

        try: details['publish'] = wait.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@class,'flex-col')]/li[1]//span[contains(@class,'break-all')]"))).text.strip()
        except:
            try: details['publish'] = driver.find_element(By.XPATH, "//div[@class='line-clamp-1 text-ellipsis font-x-small1 h-16pxr text-el-50']/span").text.strip()
            except: pass

        # 총 회차수
        try:
            # 1순위: '전체'라는 텍스트가 포함된 span 태그 찾기
            chapters_elems = driver.find_elements(By.XPATH, "//span[contains(text(), '전체')]")
            for el in chapters_elems:
                if el.text.strip().startswith("전체"):
                    details['total_chapters'] = re.sub(r'[^0-9]', '', el.text) # 숫자만 추출
                    break

            # 2순위: 혹시 태그가 숨어있다면 전체 소스에서 정규식으로 '전체 1,234' 패턴 찾기
            if not details['total_chapters']:
                match = re.search(r'전체\s*([\d,]+)', html_source)
                if match:
                    details['total_chapters'] = match.group(1).replace(',', '')
        except: pass

    except Exception as e:
        print(f"❌ 홈 탭 수집 오류: {e}")

    # 2. 정보 탭 (줄거리, 키워드)
    try:
        driver.get(base_url + "?tab_type=about")
        time.sleep(random.uniform(1.5, 2.5))

        try: details['outline'] = wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[1]/div/div[2]/div/div[1]/span'))).text.strip()
        except: pass

        try:
            keyword_elems = driver.find_elements(By.XPATH, '//*[@id="__next"]/div/div[2]/div[1]/div/div[2]/div[2]/div/div/div[2]/div[2]/div/a/div/span')
            keywords = [el.text for el in keyword_elems if el.text]
            if keywords: details['keyword'] = ", ".join(keywords)
        except: pass
    except: pass

    return details


print("\n--- 🚀 [2단계] 상세 정보 수집 시작 ---")
all_details = []
processed_details = set()

if os.path.exists(final_output_file):
    try:
        df_details_existing = pd.read_csv(final_output_file)
        all_details = df_details_existing.to_dict('records')
        processed_details = set(df_details_existing['link'])
        print(f"✅ 기존 상세 정보 {len(processed_details)}개 로드 완료.")
    except: pass

if not os.path.exists(link_output_file):
    print("🚨 수집된 링크 파일이 없습니다.")
else:
    df_all_links = pd.read_csv(link_output_file)
    links_to_scrape = list(set(df_all_links['link']) - processed_details)

    if links_to_scrape:
        print(f"✅ 수집할 새 작품: {len(links_to_scrape)}개")
        driver = None

        try:
            for i, link_url in enumerate(links_to_scrape):
                # 100개 단위로 메모리 정리를 위해 브라우저 재시작
                if i % 100 == 0:
                    if driver: driver.quit()
                    driver = start_driver()

                print(f"   ... ({i+1}/{len(links_to_scrape)}) 수집 중: {link_url}")
                details = get_novel_details(driver, link_url)
                all_details.append(details)
                time.sleep(random.uniform(1.0, 2.0))

                # 50개마다 중간 저장
                if (i + 1) % 50 == 0:
                    df_temp = pd.DataFrame(all_details)[COLUMNS_ORDER]
                    df_temp.to_csv(final_output_file, index=False, encoding="utf-8-sig")

        finally:
            print("\n🎉 모든 수집 완료!")
            if all_details:
                df_final = pd.DataFrame(all_details)[COLUMNS_ORDER]
                df_final.drop_duplicates(subset=['link'], keep='last', inplace=True)
                df_final.to_csv(final_output_file, index=False, encoding="utf-8-sig")
                print(f"💾 최종 CSV 저장 완료: {final_output_file} (총 {len(df_final)}개)")

            if driver: driver.quit()
    else:
        print("\n🎉 모든 상세 정보 수집이 이미 완료되었습니다!")


--- 🚀 [2단계] 상세 정보 수집 시작 ---
✅ 기존 상세 정보 4650개 로드 완료.
✅ 수집할 새 작품: 1178개
   ... (1/1178) 수집 중: https://page.kakao.com/content/53406956
   ... (2/1178) 수집 중: https://page.kakao.com/content/59843419
   ... (3/1178) 수집 중: https://page.kakao.com/content/65796496
   ... (4/1178) 수집 중: https://page.kakao.com/content/59703494
   ... (5/1178) 수집 중: https://page.kakao.com/content/61697988
   ... (6/1178) 수집 중: https://page.kakao.com/content/48018256
   ... (7/1178) 수집 중: https://page.kakao.com/content/62885049
   ... (8/1178) 수집 중: https://page.kakao.com/content/54545877
   ... (9/1178) 수집 중: https://page.kakao.com/content/48591273
   ... (10/1178) 수집 중: https://page.kakao.com/content/57665139
   ... (11/1178) 수집 중: https://page.kakao.com/content/49254645
   ... (12/1178) 수집 중: https://page.kakao.com/content/54247711
   ... (13/1178) 수집 중: https://page.kakao.com/content/60434327
   ... (14/1178) 수집 중: https://page.kakao.com/content/53702402
   ... (15/1178) 수집 중: https://page.kakao.com/content/63

## 전처리
- 2020년부터 2024년 작품만 필터링
- 날짜 date 타입으로 변환
- 조회수 형식 변환 (4.4만 -> 44000)
- 조회수 정규화
- 제목과 줄거리 전처리

In [ ]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/kakaopage.csv')

# 2. 날짜 전처리 (2020-2024 필터링)
df['publish_dt'] = pd.to_datetime(df['publish'], format='%y.%m.%d', errors='coerce')
df_filtered = df[(df['publish_dt'].dt.year >= 2020) & (df['publish_dt'].dt.year <= 2024)].copy()

# 3. 조회수 숫자 변환 (4.4만 -> 44000)
def to_number(x):
    if pd.isna(x): return np.nan
    x = str(x).replace(',', '').strip()

    # 1. 단위 확인 (순서 중요: 억 -> 만 -> 천)
    multiplier = 1
    if '억' in x: multiplier = 100000000
    elif '만' in x: multiplier = 10000
    elif '천' in x: multiplier = 1000

    # 2. 숫자와 소수점만 남기고 나머지는 다 제거
    num_str = "".join([c for c in x if c.isdigit() or c == '.'])

    try:
        return float(num_str) * multiplier
    except:
        return np.nan

df_filtered['views_num'] = df_filtered['views'].apply(to_number)

# 4. 조회수 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_filtered['views_normalized'] = scaler.fit_transform(np.log1p(df_filtered[['views_num']]))

print(f"✅ 필터링 전: {len(df)}개 -> 필터링 후: {len(df_filtered)}개")

df_filtered.head(10)

✅ 필터링 전: 5828개 -> 필터링 후: 3284개


,title,outline,keyword,views,publish,total_chapters,link,publish_dt,views_num,views_normalized
5,괴담 전문 사무소 :,괴이한 일에 엮이는 사람들을 도와주는 곳이 있다.\n\n“사람 사는 곳 어디에나 이...,"#미스테리, #복수, #영웅/신화, #천재, #노력",83.1만,21.10.26,269,https://page.kakao.com/content/58042144,2021-10-26,831000.0,0.572335
6,전설의 건설 재벌,이 세상에 있는 수많은 전문가들.\n그들 모두가 내 멘토다.\n\n[계승을 시작합니...,NaN,7만,21.12.01,151,https://page.kakao.com/content/58321168,2021-12-01,70000.0,0.414923
8,던전에서 초월급 인공지능을 주웠다,능력치 최하의 짐꾼이 신이 만든 인공지능을 주웠다!\n둘의 목표는 던전 정복!\n임...,"#AI, #조력자",2.9만,24.10.10,250,https://page.kakao.com/content/65153154,2024-10-10,29000.0,0.358859
9,천재작가는 작품의 미래를 본다,억울하게 밀려나 사고를 당한 매니저가 괴물 천재작가로 돌아왔다.\n그에게 작품과 배...,"#전문직, #천재, #작가, #방송, #PD/감독, #매니저, #배우",13.6만,24.10.07,150,https://page.kakao.com/content/65177121,2024-10-07,136000.0,0.457178
12,아포칼립스 인 아일랜드,[라스트 아일랜드]\n섬에 존재하는 여러 생태계의 돌연변이 동식물과 유저 시스템을 ...,"#좀비, #생존, #아포칼립스, #시스템",1.7만,24.07.05,201,https://page.kakao.com/content/64509995,2024-07-05,17000.0,0.324880
13,전설의 축구감독 [완결],EPL 명문 구단의 감독이었던 나.\n눈을 떴을 때는 15년 전의 과거로 돌아와 있...,"#스포츠, #회귀, #성장, #축구, #PD/감독",27.4만,21.04.20,226,https://page.kakao.com/content/56920653,2021-04-20,274000.0,0.501745
14,귀신 잡는 사장님,"“살아봐.”\n“네? 그게 무슨…….”\n""작은 행복이 담긴 삶을 네 손으로 만들어...",NaN,5.6만,20.07.19,221,https://page.kakao.com/content/55327890,2020-07-19,56000.0,0.400726
15,재벌서자이지만 천재라서 [완결],삶의 끝과 회귀를 관통하며 얻은 천재적 능력\n덕분에 재벌가의 재산 따윈 필요 없었...,NaN,120.1만,20.03.02,185,https://page.kakao.com/content/54523831,2020-03-02,1201000.0,0.595766
17,인생역전 우강호,하루하루 근근이 살아가던 흙수저 취준생 우강호.\n\n그에게 우연처럼 찾아온 단 한...,"#경영/기업, #재벌",27.5만,24.02.27,375,https://page.kakao.com/content/63682010,2024-02-27,275000.0,0.501976
19,인생 2회차 재벌 3세는 모든 게 잘 풀린다,"""한 개인이 지속 가능한 위대한 기업을 만들 수는 없다. 하지만 그 반대의 경우 권...","#경영/기업, #복수, #회귀",58.1만,23.03.10,200,https://page.kakao.com/content/61328033,2023-03-10,581000.0,0.549565


In [ ]:
import re

### 제목 전처리 ###
def clean_title(text):
    if pd.isna(text): return ""

    # 1. 괄호와 그 안의 내용 제거 (회차 정보, 독점, 공모전 등)
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)

    # 2. 제목 끝에 흔히 붙는 불필요한 기호 및 공백 제거
    text = text.strip().rstrip('.')

    # 3. 연속된 공백 하나로 통합
    text = re.sub(r'\s+', ' ', text)

    return text

### 줄거리 전처리 ###
import pandas as pd
import re

def clean_outline(row):
    # 빈 값 처리
    outline = str(row['outline']) if pd.notna(row['outline']) else ""
    keyword_col = str(row['keyword']) if pd.notna(row['keyword']) else ""

    # 1. 줄거리에서 키워드 추출 및 병합
    tags_in_outline = re.findall(r'#([가-힣a-zA-Z0-9_]+)', outline)
    tags_in_keyword = re.findall(r'#([가-힣a-zA-Z0-9_]+)', keyword_col)

    combined_tags = set(tags_in_outline + tags_in_keyword)
    final_keywords = ", ".join(sorted([tag for tag in combined_tags if tag]))


    # 2. 줄거리에서 해시태그 부분 완전히 삭제
    text = re.sub(r'#[가-힣a-zA-Z0-9_]+', '', outline)

    # 3. 불필요 문구 제거
    noise_patterns = [
        r'[-=~]{3,}',                  # ---------------- 같은 긴 구분선 제거
        r'<.*?>\s*개정판\s*입니다.*',  # <제목> 개정판 입니다~ 형식의 공지 제거
        r'참고\s*바랍니다.*',          # 안내 멘트 제거
        r'ⓒ.*',                       # 저작권 표기 제거
        r'\[.*?연재.*?\]'              # [독점 연재] 같은 홍보용 대괄호 제거
    ]
    for pattern in noise_patterns:
        text = re.sub(pattern, ' ', text, flags=re.IGNORECASE)

    # 4. 특수문자 정제 (상태창을 나타내는 [], <> 는 웹소설 핵심이므로 보존!)
    text = re.sub(r'[^가-힣0-9a-zA-Z\s\.\?\!,\'\"\[\]\<\>\(\)]', ' ', text)

    # 5. 다중 공백(스페이스바 여러 개, 줄바꿈 등)을 띄어쓰기 하나로 압축
    text = re.sub(r'\s+', ' ', text).strip()

    return text, final_keywords


df_filtered[['outline_clean', 'keyword_final']] = df_filtered.apply(
    lambda row: pd.Series(clean_outline(row)), axis=1)
df_filtered['title_clean'] = df_filtered['title'].apply(clean_title)

df_filtered.head(10)

,title,outline,keyword,views,publish,total_chapters,link,publish_dt,views_num,views_normalized,outline_clean,keyword_final,title_clean
5,괴담 전문 사무소 :,괴이한 일에 엮이는 사람들을 도와주는 곳이 있다.\n\n“사람 사는 곳 어디에나 이...,"#미스테리, #복수, #영웅/신화, #천재, #노력",83.1만,21.10.26,269,https://page.kakao.com/content/58042144,2021-10-26,831000.0,0.572335,괴이한 일에 엮이는 사람들을 도와주는 곳이 있다. 사람 사는 곳 어디에나 이상한 이...,"노력, 미스테리, 복수, 영웅, 천재",괴담 전문 사무소 :
6,전설의 건설 재벌,이 세상에 있는 수많은 전문가들.\n그들 모두가 내 멘토다.\n\n[계승을 시작합니...,NaN,7만,21.12.01,151,https://page.kakao.com/content/58321168,2021-12-01,70000.0,0.414923,이 세상에 있는 수많은 전문가들. 그들 모두가 내 멘토다. [계승을 시작합니다.] ...,,전설의 건설 재벌
8,던전에서 초월급 인공지능을 주웠다,능력치 최하의 짐꾼이 신이 만든 인공지능을 주웠다!\n둘의 목표는 던전 정복!\n임...,"#AI, #조력자",2.9만,24.10.10,250,https://page.kakao.com/content/65153154,2024-10-10,29000.0,0.358859,능력치 최하의 짐꾼이 신이 만든 인공지능을 주웠다! 둘의 목표는 던전 정복! 임선호...,"AI, 조력자",던전에서 초월급 인공지능을 주웠다
9,천재작가는 작품의 미래를 본다,억울하게 밀려나 사고를 당한 매니저가 괴물 천재작가로 돌아왔다.\n그에게 작품과 배...,"#전문직, #천재, #작가, #방송, #PD/감독, #매니저, #배우",13.6만,24.10.07,150,https://page.kakao.com/content/65177121,2024-10-07,136000.0,0.457178,억울하게 밀려나 사고를 당한 매니저가 괴물 천재작가로 돌아왔다. 그에게 작품과 배우...,"PD, 매니저, 방송, 배우, 작가, 전문직, 천재",천재작가는 작품의 미래를 본다
12,아포칼립스 인 아일랜드,[라스트 아일랜드]\n섬에 존재하는 여러 생태계의 돌연변이 동식물과 유저 시스템을 ...,"#좀비, #생존, #아포칼립스, #시스템",1.7만,24.07.05,201,https://page.kakao.com/content/64509995,2024-07-05,17000.0,0.324880,[라스트 아일랜드] 섬에 존재하는 여러 생태계의 돌연변이 동식물과 유저 시스템을 활...,"생존, 시스템, 아포칼립스, 좀비",아포칼립스 인 아일랜드
13,전설의 축구감독 [완결],EPL 명문 구단의 감독이었던 나.\n눈을 떴을 때는 15년 전의 과거로 돌아와 있...,"#스포츠, #회귀, #성장, #축구, #PD/감독",27.4만,21.04.20,226,https://page.kakao.com/content/56920653,2021-04-20,274000.0,0.501745,EPL 명문 구단의 감독이었던 나. 눈을 떴을 때는 15년 전의 과거로 돌아와 있었...,"PD, 성장, 스포츠, 축구, 회귀",전설의 축구감독
14,귀신 잡는 사장님,"“살아봐.”\n“네? 그게 무슨…….”\n""작은 행복이 담긴 삶을 네 손으로 만들어...",NaN,5.6만,20.07.19,221,https://page.kakao.com/content/55327890,2020-07-19,56000.0,0.400726,"살아봐. 네? 그게 무슨 . ""작은 행복이 담긴 삶을 네 손으로 만들어보라고."" 지...",,귀신 잡는 사장님
15,재벌서자이지만 천재라서 [완결],삶의 끝과 회귀를 관통하며 얻은 천재적 능력\n덕분에 재벌가의 재산 따윈 필요 없었...,NaN,120.1만,20.03.02,185,https://page.kakao.com/content/54523831,2020-03-02,1201000.0,0.595766,삶의 끝과 회귀를 관통하며 얻은 천재적 능력 덕분에 재벌가의 재산 따윈 필요 없었다...,,재벌서자이지만 천재라서
17,인생역전 우강호,하루하루 근근이 살아가던 흙수저 취준생 우강호.\n\n그에게 우연처럼 찾아온 단 한...,"#경영/기업, #재벌",27.5만,24.02.27,375,https://page.kakao.com/content/63682010,2024-02-27,275000.0,0.501976,하루하루 근근이 살아가던 흙수저 취준생 우강호. 그에게 우연처럼 찾아온 단 한 번의...,"경영, 재벌",인생역전 우강호
19,인생 2회차 재벌 3세는 모든 게 잘 풀린다,"""한 개인이 지속 가능한 위대한 기업을 만들 수는 없다. 하지만 그 반대의 경우 권...","#경영/기업, #복수, #회귀",58.1만,23.03.10,200,https://page.kakao.com/content/61328033,2023-03-10,581000.0,0.549565,"""한 개인이 지속 가능한 위대한 기업을 만들 수는 없다. 하지만 그 반대의 경우 권...","경영, 복수, 회귀",인생 2회차 재벌 3세는 모든 게 잘 풀린다


In [ ]:
df_filtered.info()
df_filtered.to_csv('/content/drive/MyDrive/Colab Notebooks/kakao_cleaned.csv')

<class 'pandas.core.frame.DataFrame'>
Index: 3284 entries, 5 to 5825
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             3284 non-null   object        
 1   outline           3284 non-null   object        
 2   keyword           2562 non-null   object        
 3   views             3284 non-null   object        
 4   publish           3284 non-null   object        
 5   total_chapters    3284 non-null   int64         
 6   link              3284 non-null   object        
 7   publish_dt        3284 non-null   datetime64[ns]
 8   views_num         3284 non-null   float64       
 9   views_normalized  3284 non-null   float64       
 10  outline_clean     3284 non-null   object        
 11  keyword_final     3284 non-null   object        
 12  title_clean       3284 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(9)
memory usage: 488.2+ KB


# 네이버시리즈

## 크롤링
- 전체회차 수와 연재시작일은 동적 요소로, 나눠서 수집 진행

In [ ]:
!pip install selenium pandas

# 1. 크롬 브라우저 & 드라이버 다운로드 (코랩 리눅스 환경용)
!wget -q https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chrome-linux64.zip
!wget -q https://storage.googleapis.com/chrome-for-testing-public/122.0.6261.94/linux64/chromedriver-linux64.zip

# 2. 압축 해제 (코드에서 찾는 /content/chromedriver-linux64/ 경로가 자동 생성됨)
!unzip -o -q chrome-linux64.zip
!unzip -o -q chromedriver-linux64.zip

# 3. 리눅스 실행 권한 부여 (매우 중요!)
!chmod +x /content/chromedriver-linux64/chromedriver
!chmod +x /content/chrome-linux64/chrome

print("✅ 크롬 브라우저 및 드라이버 셋업 완료!")

✅ 크롬 브라우저 및 드라이버 셋업 완료!


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import random

# ===== 기본 설정 =====
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
csv_file = "/content/drive/MyDrive/Colab Notebooks/naver.csv"
save_interval = 100  # 100개마다 중간 저장
all_data = []

# ===== 기존 CSV 불러오기 =====
existing_links = set()
if os.path.exists(csv_file):
    df_existing = pd.read_csv(csv_file)
    all_data = df_existing.to_dict("records")
    existing_links = set(df_existing["link"])
    print(f"기존 데이터 {len(all_data)}개 불러옴")

# ===== 상세 정보 수집 함수 =====
def get_details(link):
    details = {'views': None, 'outline': None}
    try:
        res = requests.get(link, headers=headers, timeout=10)
        if res.status_code == 200:
            soup = BeautifulSoup(res.text, "lxml")

            # 1. 총 다운로드 수(조회수) 추출
            download_tag = soup.select_one("a.btn_download span")
            if download_tag:
                details['views'] = download_tag.get_text(strip=True)

            # 2. 줄거리(시놉시스) 추출
            synopsis_tags = soup.select("div._synopsis")
            if len(synopsis_tags) > 1:
                details['outline'] = synopsis_tags[1].get_text(separator=" ", strip=True)
            elif synopsis_tags:
                details['outline'] = synopsis_tags[0].get_text(separator=" ", strip=True)

    except Exception as e:
        print(f"   > 요청 오류: {e}")
    return details

# ===== 본문 크롤링 =====
print(f"\n=== 1차 수집 시작 ===")
page = 1
prev_titles = set()

while True:
    url = f"https://series.naver.com/novel/categoryProductList.series?categoryTypeCode=genre&genreCode=208&page={page}"
    print(f"[페이지 {page}] 수집 중...")

    try:
        res = requests.get(url, headers=headers, timeout=10)
        if res.status_code != 200:
            break

        soup = BeautifulSoup(res.text, "lxml")
        novels = soup.select("ul.lst_list li")
        if not novels:
            break

        current_titles = set()
        for n in novels:
            title_tag = n.select_one("h3 a")
            if not title_tag: continue

            title = title_tag.get_text(strip=True)
            if "단행본" in title: continue # 단행본 제외

            link = "https://series.naver.com" + title_tag.get("href")
            current_titles.add(title)

            if link in existing_links: continue

            details = get_details(link)

            # 데이터 딕셔너리 구조 통일 (영문 키 사용 권장)
            all_data.append({
                "title": title,
                "outline": details['outline'],
                "views": details['views'],
                "publish": None,      # 2차 Selenium에서 채움
                "total_chapters": None, # 2차 Selenium에서 채움
                "link": link
            })
            existing_links.add(link)

            if len(all_data) % save_interval == 0:
                df = pd.DataFrame(all_data).drop_duplicates(subset=["title", "link"])
                df.to_csv(csv_file, index=False, encoding="utf-8-sig")
                print(f"중간 저장 완료: {len(df)}개")

        if current_titles == prev_titles:
            print("마지막 페이지 반복 감지. 수집 종료.")
            break

        prev_titles = current_titles
        page += 1
        time.sleep(random.uniform(1, 2))

    except Exception as e:
        print(f"크롤링 오류: {e}")
        break

# 최종 저장
df = pd.DataFrame(all_data).drop_duplicates(subset=["title", "link"])
df.to_csv(csv_file, index=False, encoding="utf-8-sig")
print(f"1차 수집 완료: 총 {len(df)}개 작품")

KeyboardInterrupt: 

In [ ]:
import pandas as pd
import re, time, random, sys
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Chrome 세팅 (경로는 본인 환경에 맞게 유지)
DRIVER_PATH = '/content/chromedriver-linux64/chromedriver'
BROWSER_PATH = '/content/chrome-linux64/chrome'

chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.binary_location = BROWSER_PATH
service = Service(DRIVER_PATH)
driver = webdriver.Chrome(service=service, options=chrome_options)

csv_file = "/content/drive/MyDrive/Colab Notebooks/naver.csv"
df = pd.read_csv(csv_file)

# 수집해야 할 데이터 필터링
df_missing = df[df["publish"].isna()]
print(f"보완이 필요한 작품 수: {len(df_missing)}")

def get_detail_info_selenium(link):
    """전체 회차 수와 첫 화 연재일(YYYY.MM.DD) 추출"""
    info = {'chapters': None, 'date': None}
    try:
        # 정렬을 과거순(ASC)으로 변경하여 1화가 맨 위에 오도록 설정
        if "sortOrder" not in link:
            target_url = link + "&sortOrder=ASC" if "?" in link else link + "?sortOrder=ASC"
        else:
            target_url = link

        driver.get(target_url)
        wait = WebDriverWait(driver, 5)

        # 1. 총 회차수 수집
        try:
            # 안전하게 요소가 뜰 때까지 대기
            chapter_elem = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#content > h5 > strong")))
            info['chapters'] = re.sub(r'[^0-9]', '', chapter_elem.text)
        except Exception as e:
            print(f" 회차 에러: {e}")
            pass

        # 2. 첫 화 연재일 추출 (정확한 날짜 형태 YYYY.MM.DD)
        try:
            # 페이지 구조에 따라 날짜 텍스트 위치가 다를 수 있으므로 텍스트 전체에서 날짜 패턴 탐색
            first_row = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#volumeList tr._volume_row_1")))
            page_text = first_row.text
            # YYYY.MM.DD 형식 찾기
            date_match = re.search(r"(\d{4}\.\d{2}\.\d{2})", page_text)
            if date_match:
                info['date'] = date_match.group(1)
        except Exception as e:
            print(f" 날짜 에러: {e}")
            pass

    except Exception as e:
        print(f"전체 에러: {e}")
        pass

    return info

# 실행부
updated_count = 0
for idx, row in df_missing.iterrows():
    link = row["link"]

    extracted = get_detail_info_selenium(link)

    # 🚨 [수정] 실행부 칼럼명 매칭 ('first_date' -> 'publish')
    if extracted['date']:
        df.at[idx, "publish"] = extracted['date']
    if extracted['chapters']:
        df.at[idx, "total_chapters"] = extracted['chapters']

    print(f"[{updated_count+1}] {row['title'][:10]}... | 회차: {extracted['chapters']} | 날짜: {extracted['date']}")

    updated_count += 1
    time.sleep(random.uniform(1.0, 2.5))

    if updated_count % 50 == 0:
        df.to_csv(csv_file, index=False, encoding="utf-8-sig")

df.to_csv(csv_file, index=False, encoding="utf-8-sig")
driver.quit()
print("상세 정보 보완 완료!")

보완이 필요한 작품 수: 6526


/tmp/ipython-input-2213954709.py:79: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2026.02.05' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "publish"] = extracted['date']
/tmp/ipython-input-2213954709.py:81: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '118' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.at[idx, "total_chapters"] = extracted['chapters']


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[2100] 더 기븐(THE G... | 회차: 100 | 날짜: 2024.06.28
[2101] 이혼 후 작두 탔습... | 회차: 240 | 날짜: 2024.02.16
[2102] 슬기로운 천마님의 ... | 회차: 175 | 날짜: 2024.03.18
[2103] 이름대로 살련다(1... | 회차: 151 | 날짜: 2023.12.06
[2104] 능력흡수로 무한성장... | 회차: 114 | 날짜: 2024.06.27
[2105] 건달이 각성하면 벌... | 회차: 250 | 날짜: 2024.01.25
[2106] 벤츠 타는 마법사(... | 회차: 161 | 날짜: 2024.06.27
[2107] 이번 앨범의 컨셉은... | 회차: 178 | 날짜: 2024.01.30
[2108] 미국 깡촌의 천재 ... | 회차: 415 | 날짜: 2023.05.26
[2109] 빙하기 아포칼립스의... | 회차: 235 | 날짜: 2023.11.08
[2110] 탑스타 버리고 회귀... | 회차: 160 | 날짜: 2024.04.22
[2111] 내 회귀에 빌런은 ... | 회차: 202 | 날짜: 2024.05.28
[2112] 핏빛 로맨스(59화... | 회차: 59 | 날짜: 2024.06.24
[2113] 회귀 경찰의 리셋 ... | 회차: 1039 | 날짜: 2024.06.24
[2114] 격투의 제왕 [독점... | 회차: 212 | 날짜: 2022.11.23
[2115] 초고교급 순정파 스... | 회차: 204 | 날짜: 2024.03.15
[2116] 아포칼립스 N수생(... | 회차: 245 | 날짜: 2023.11.14
[2117] 용 울음 드리운(1... | 회차: 101 | 날짜: 2024.06.21
[2118] 집밥을 너무 잘함(... | 회차: 150 | 날짜: 2024.06.21
[2119] 예술병 영화감독이 ... | 회차: 200 | 날짜: 2023.11.17
[212

## 전처리
대부분 네이버시리즈와 동일
- 카카오페이지는 키워드를 따로 제공하지만, 일부 소설에서 줄거리에 #키워드의 형식이 보임 -> 해당 부분을 줄거리 열에 입력

In [ ]:
import pandas as pd
import numpy as np
import re

# 1. 데이터 로드
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/naver.csv')

# 2. 날짜 전처리 (2020-2024 필터링)
df['publish_dt'] = pd.to_datetime(df['publish'], errors='coerce')
df_filtered = df[(df['publish_dt'].dt.year >= 2020) & (df['publish_dt'].dt.year <= 2024)].copy()

# 3. 조회수 숫자 변환 (4.4만 -> 44000)
def to_number(x):
    if pd.isna(x): return np.nan
    x = str(x).replace(',', '').strip()

    # 1. 단위 확인 (순서 중요: 억 -> 만 -> 천)
    multiplier = 1
    if '억' in x: multiplier = 100000000
    elif '만' in x: multiplier = 10000
    elif '천' in x: multiplier = 1000

    # 2. 숫자와 소수점만 남기고 나머지는 다 제거
    num_str = "".join([c for c in x if c.isdigit() or c == '.'])

    try:
        return float(num_str) * multiplier
    except:
        return np.nan

df_filtered['views_num'] = df_filtered['views'].apply(to_number)

# 4. 조회수 정규화
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
df_filtered['views_normalized'] = scaler.fit_transform(np.log1p(df_filtered[['views_num']]))

print(f"✅ 필터링 전: {len(df)}개 -> 필터링 후: {len(df_filtered)}개")

df_filtered.head(10)

✅ 필터링 전: 6526개 -> 필터링 후: 3785개


,title,outline,views,publish,total_chapters,link,publish_dt,views_num,views_normalized
1,나의 유럽 원정기(491화/미완결),[2024 지상최대 웹소설 공모전 수상작] 크로아티아에 배낭 여행을 왔다가 지진과 ...,175.3만,2024.09.24,491.0,https://series.naver.com/novel/detail.series?p...,2024-09-24,1753000.0,0.672170
5,부업으로 할리우드 스타합니다.(310화/미완결),악당과 마약이 난무하고 범죄율은 사상 최악을 찍은 1990년대 미국. 나는 그곳으로...,21.4만,2024.09.09,310.0,https://series.naver.com/novel/detail.series?p...,2024-09-09,214000.0,0.482879
18,괴물 천재투수가 메이저리그를 찢음(719화/미완결),나는 그저 범재일뿐이었다 하지만 내게 주어진 무한한 시간이 그 재능의 한계를 찢어버...,558.1만,2024.11.13,719.0,https://series.naver.com/novel/detail.series?p...,2024-11-13,5581000.0,0.776401
21,재벌 3세는 조용히 살고 싶다(2187화/미완결),KM 그룹의 막내 최병문의 서자 최민혁은 아버지의 죽음 이후 어머니 정미선을 떠나 ...,172.5만,2020.09.16,2187.0,https://series.naver.com/novel/detail.series?p...,2020-09-16,1725000.0,0.670721
27,법보다 주먹 [개정판](788화/완결),법으로 안되면 주먹! 단순한 것이 진리가 되는 세상 법보다 주먹 접기,59.7만,2022.03.16,788.0,https://series.naver.com/novel/detail.series?p...,2022-03-16,597000.0,0.575219
29,범상한 변호사의 아공간(445화/미완결),대한민국 최고 로펌에 입사했다.,519.5만,2024.04.11,445.0,https://series.naver.com/novel/detail.series?p...,2024-04-11,5195000.0,0.769950
32,수술실 온 에어(2518화/미완결),*이 작품은 眞熊初墨 작가의 소설 手術直播間(2018)을 한국어로 옮긴 것입니다. ...,26.9만,2020.08.14,2518.0,https://series.naver.com/novel/detail.series?p...,2020-08-14,269000.0,0.503466
40,1953 맨손 재벌(827화/미완결),먼저 간 아내와 아들을 만나기 위해 1953년으로 회귀했다. 처음부터 재벌이 될 생...,307.2만,2023.02.24,827.0,https://series.naver.com/novel/detail.series?p...,2023-02-24,3072000.0,0.722664
43,허건필 센타 [독점](378화/미완결),허건필 센타 - 제령/퇴마/구마 써비스 전문! ※귀신이 보이시는 분 ※귀신에게 시달...,104.8만,2023.10.10,378.0,https://series.naver.com/novel/detail.series?p...,2023-10-10,1048000.0,0.625868
45,2회차 천재 디자이너 [독점](441화/미완결),디자이너 2회차. 사람들의 호감도가 보이기 시작했다. [2021 지상최대공모전 수상...,132.6만,2024.07.16,441.0,https://series.naver.com/novel/detail.series?p...,2024-07-16,1326000.0,0.647044


In [ ]:
import re

### 제목 전처리 ###
def clean_title(text):
    if pd.isna(text): return ""

    # 1. 괄호와 그 안의 내용 제거 (회차 정보, 독점, 공모전 등)
    text = re.sub(r'\[.*?\]|\(.*?\)', '', text)

    # 2. 제목 끝에 흔히 붙는 불필요한 기호 및 공백 제거
    text = text.strip().rstrip('.')

    # 3. 연속된 공백 하나로 통합
    text = re.sub(r'\s+', ' ', text)

    return text

### 줄거리 전처리 ###
def clean_outline(text):
    if pd.isna(text): return ""

    # 1. 네이버 특유의 '접기' 버튼 글자 제거
    text = text.replace('접기', '')

    # 2. 문장 맨 앞/뒤의 홍보용 대괄호 태그만 제거
    promo_patterns = [
        r'\[.*?(공모전|수상|독점|선공개|완결|개정판|단행본).*?\]',
        r'^\*이\s작품은.*?것입니다\.', # 번역물 안내 문구
    ]
    for p in promo_patterns:
        text = re.sub(p, '', text, flags=re.IGNORECASE).strip()

    # 3. 줄거리 본문과 무관한 하단 메타 정보 제거 (작가/출판사/문의 등)
    meta_patterns = [
        r'작가\s?:\s?\w+',
        r'출판사\s?:\s?\w+',
        r'문의\s?:\s?\S+',
        r'<작가\s?소개>.*', # <작가 소개> 뒤로는 다 날림
        r'ⓒ.*'
    ]
    for p in meta_patterns:
        text = re.sub(p, '', text, flags=re.IGNORECASE | re.DOTALL)

    # 4. 특수문자 정제 (한글, 숫자, 영문, 주요 문장부호 유지)
    text = re.sub(r'[^가-힣0-9a-zA-Z\s\.\?\!]', ' ', text)

    # 5. 연속된 공백 정리
    text = re.sub(r'\s+', ' ', text).strip()

    return text

# 적용
df_filtered['outline_clean'] = df_filtered['outline'].apply(clean_outline)
df_filtered['title_clean'] = df_filtered['title'].apply(clean_title)

df_filtered.head(5)

,title,outline,views,publish,total_chapters,link,publish_dt,views_num,views_normalized,outline_clean,title_clean
1,나의 유럽 원정기(491화/미완결),[2024 지상최대 웹소설 공모전 수상작] 크로아티아에 배낭 여행을 왔다가 지진과 ...,175.3만,2024.09.24,491.0,https://series.naver.com/novel/detail.series?p...,2024-09-24,1753000.0,0.672170,크로아티아에 배낭 여행을 왔다가 지진과 코로나로 인해 발이 묶여버린 게 원인이었다....,나의 유럽 원정기
5,부업으로 할리우드 스타합니다.(310화/미완결),악당과 마약이 난무하고 범죄율은 사상 최악을 찍은 1990년대 미국. 나는 그곳으로...,21.4만,2024.09.09,310.0,https://series.naver.com/novel/detail.series?p...,2024-09-09,214000.0,0.482879,악당과 마약이 난무하고 범죄율은 사상 최악을 찍은 1990년대 미국. 나는 그곳으로...,부업으로 할리우드 스타합니다
18,괴물 천재투수가 메이저리그를 찢음(719화/미완결),나는 그저 범재일뿐이었다 하지만 내게 주어진 무한한 시간이 그 재능의 한계를 찢어버...,558.1만,2024.11.13,719.0,https://series.naver.com/novel/detail.series?p...,2024-11-13,5581000.0,0.776401,나는 그저 범재일뿐이었다 하지만 내게 주어진 무한한 시간이 그 재능의 한계를 찢어버렸다,괴물 천재투수가 메이저리그를 찢음
21,재벌 3세는 조용히 살고 싶다(2187화/미완결),KM 그룹의 막내 최병문의 서자 최민혁은 아버지의 죽음 이후 어머니 정미선을 떠나 ...,172.5만,2020.09.16,2187.0,https://series.naver.com/novel/detail.series?p...,2020-09-16,1725000.0,0.670721,KM 그룹의 막내 최병문의 서자 최민혁은 아버지의 죽음 이후 어머니 정미선을 떠나 ...,재벌 3세는 조용히 살고 싶다
27,법보다 주먹 [개정판](788화/완결),법으로 안되면 주먹! 단순한 것이 진리가 되는 세상 법보다 주먹 접기,59.7만,2022.03.16,788.0,https://series.naver.com/novel/detail.series?p...,2022-03-16,597000.0,0.575219,법으로 안되면 주먹! 단순한 것이 진리가 되는 세상 법보다 주먹,법보다 주먹


In [ ]:
df_filtered.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3785 entries, 1 to 5224
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             3785 non-null   object        
 1   outline           3785 non-null   object        
 2   views             3741 non-null   object        
 3   publish           3785 non-null   object        
 4   total_chapters    3785 non-null   float64       
 5   link              3785 non-null   object        
 6   publish_dt        3785 non-null   datetime64[ns]
 7   views_num         3741 non-null   float64       
 8   views_normalized  3741 non-null   float64       
 9   outline_clean     3785 non-null   object        
 10  title_clean       3785 non-null   object        
dtypes: datetime64[ns](1), float64(3), object(7)
memory usage: 483.9+ KB


In [ ]:
# 조회수가 아예 수집되지 않은 44개 작품 삭제
df_filtered = df_filtered.dropna(subset=['views_num']).copy()

print(f"완벽하게 정제된 최종 데이터 개수: {len(df_filtered)}개")

완벽하게 정제된 최종 데이터 개수: 3741개


In [ ]:
df_filtered.to_csv('/content/drive/MyDrive/Colab Notebooks/naver_cleaned.csv', index=False)

# 파일 결합
동일한 작품이 양 플랫폼에 존재할 경우 아래의 기준 적용
- 연재시작일: 빠른 날짜
- 조회수: 평균
- 줄거리: 유사도가 70% 이상일 경우 긴 줄거리 하나만
- 전체회차: 높은 숫자

In [ ]:
import pandas as pd
import re
import difflib

naver = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/naver_cleaned.csv")
kakao = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/kakao_cleaned.csv")

# 1. 네이버와 카카오 데이터 통합
cols = ['title_clean', 'outline_clean', 'views_num', 'publish_dt','total_chapters', 'keyword_final']
if 'keyword_final' not in naver.columns:
    naver['keyword_final'] = ""

df_total = pd.concat([naver[cols], kakao[cols]], ignore_index=True)

# 2. 유사도 기반 줄거리 병합
def merge_outlines(outlines):
    valid_texts = [str(x).strip() for x in outlines if pd.notna(x) and str(x).strip()]

    # 겹치는 소설이 없어서 줄거리가 1개뿐이면 그냥 그거 반환
    if len(valid_texts) == 1:
        return valid_texts[0]

    # 양쪽 플랫폼에 모두 있어서 줄거리가 2개 이상인 경우
    elif len(valid_texts) >= 2:
        text1 = valid_texts[0]
        text2 = valid_texts[1]

        # 두 텍스트의 유사도 계산 (0.0 ~ 1.0)
        similarity = difflib.SequenceMatcher(None, text1, text2).ratio()

        if similarity >= 0.70:
            # 70% 이상 비슷하면 거의 같은 줄거리 -> 글자수가 더 긴 것을 선택
            return max(valid_texts, key=len)
        else:
            # 70% 미만이면 완전히 다른 소개글 -> 두 개를 합쳐버림! (정보 손실 방지)
            return text1 + " / [추가 줄거리] " + text2

    return ""

# 3. 키워드 병합
def combine_keywords(keywords):
    combined = set()
    for k in keywords:
        if pd.notna(k) and str(k).strip():
            # 쉼표로 쪼개서 세트에 넣음 (자동 중복 제거)
            combined.update([tag.strip() for tag in str(k).split(',') if tag.strip()])
    # 다시 가나다순 정렬 후 콤마로 연결
    return ", ".join(sorted(list(combined)))

# 4. 적용하기
# 비교용 제목
df_total['title_compare'] = df_total['title_clean'].astype(str).str.replace(' ', '', regex=False)

df_merged = df_total.groupby('title_compare').agg({
    'title_clean': 'first',
    'publish_dt': 'min',
    'total_chapters': 'max',
    'views_num': 'mean',
    'outline_clean': merge_outlines,
    'keyword_final': combine_keywords
}).reset_index(drop=True)

print("✨ 지능형 줄거리 병합 완료!")

df_merged.info()

✨ 지능형 줄거리 병합 완료!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title_clean     4308 non-null   object 
 1   publish_dt      4308 non-null   object 
 2   total_chapters  4308 non-null   float64
 3   views_num       4308 non-null   float64
 4   outline_clean   4308 non-null   object 
 5   keyword_final   4308 non-null   object 
dtypes: float64(2), object(4)
memory usage: 202.1+ KB


In [ ]:
# 1. 날짜 데이터 타입 복구 (object -> datetime)
df_merged['publish_dt'] = pd.to_datetime(df_merged['publish_dt'])

# 2. 원하시는 순서대로 컬럼 리스트 작성
new_order = [
    'title_clean',       # 제목
    'outline_clean',     # 줄거리
    'keyword_final',     # 키워드
    'views_num',  # 조회수
    'publish_dt',        # 연재시작일
    'total_chapters'     # 전체회차
]

# 3. 데이터프레임 컬럼 순서 재배치
df_merged = df_merged[new_order]

# 4. 파생변수
base_date = pd.to_datetime('2026-02-20')

# 연재 후 흐른 시간 (통제 변수용)
df_merged['days_passed'] = (base_date - df_merged['publish_dt']).dt.days
# 회당 조회수 (종속 변수용)
df_merged['views_per_chapter'] = df_merged['views_num'] / df_merged['total_chapters']

df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4308 entries, 0 to 4307
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   title_clean        4308 non-null   object        
 1   outline_clean      4308 non-null   object        
 2   keyword_final      4308 non-null   object        
 3   views_num          4308 non-null   float64       
 4   publish_dt         4308 non-null   datetime64[ns]
 5   total_chapters     4308 non-null   float64       
 6   days_passed        4308 non-null   int64         
 7   views_per_chapter  4308 non-null   float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(3)
memory usage: 269.4+ KB


/tmp/ipython-input-3084529154.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged['publish_dt'] = pd.to_datetime(df_merged['publish_dt'])


In [ ]:
df_merged.to_csv("/content/drive/MyDrive/Colab Notebooks/merged.csv")